<a href="https://colab.research.google.com/github/Mrinal18/AI_for_Security/blob/main/PATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pate

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

n_instances = 500 
n_teachers = 3

# load data and transform it for cifar100
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( np.float32 ) / 255
x_test = x_test.astype( np.float32 ) / 255

x_train = x_train.reshape( -1, 32, 32, 3 )
x_test = x_test.reshape( -1, 32, 32, 3 )

y_train = keras.utils.to_categorical( y_train )
y_test = keras.utils.to_categorical( y_test )

# shuffle data
idx = np.random.permutation( x_train.shape[ 0 ] )
np.random.seed( 42 )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = [ x_train[ i * n_instances : (i + 1) * n_instances ] for i in range( n_teachers ) ]
teacher_data_y = [ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ]

# gather the student data
student_data_x = x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  
student_data_y = y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  


In [ ]:
## train the teacher models
# train the teacher models
def get_model():
    model = keras.models.Sequential()
    model.add( keras.layers.Conv2D( 32, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[ 1: ]  ) )
    model.add( keras.layers.MaxPooling2D( pool_size=(2, 2) ) )
    model.add( keras.layers.Conv2D( 64, kernel_size=(3, 3), activation='relu' ) )
    model.add( keras.layers.Flatten() )
    model.add( keras.layers.Dense( 128, activation='relu' ) )
    model.add( keras.layers.Dense( 10, activation='softmax' ) )
    return model

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

# train teacher models
for i in range( n_teachers ):
    teacher_models[ i ].compile( loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'] )
    teacher_models[ i ].fit( teacher_data_x[ i ], teacher_data_y[ i ], batch_size=128, epochs=10, verbose=1, validation_data=( x_test, y_test ) )